## Geração complementar de códigos de validação de votos de evento gastronômico, considerando os códigos já gerados anteriormente

### Imports ->

In [16]:
import pandas as pd
import numpy as np
import string
import secrets

### Carregamento dos códigos anteriores ->

In [17]:
# Carregamento dos códigos gerados originalmente em data frame pandas
df_codigos_antigos = pd.read_excel(r"codigos-categorias-geral.xlsx")

# Criando lista a partir do data frame
tabela_codigos = list(df_codigos_antigos['Código'])

### Funções de geração de códigos (adaptada para novos códigos) e de exportação para excel ->

In [18]:
def gerar_codigos(quantidade, tamanho=6, letras_maiusculas=False, letras_minusculas=True, numeros=True, simbolos=False, prefixo='', contador=0, codigos_existentes=[]):
    '''Gera códigos aleatórios únicos
    
    Parameters:
        quantidade (inteiro): Quantidade de códigos aleatórios a serem gerados
        tamanho (inteiro): Quantidade de caracteres para cada código
        letras_maiusculas (boolean): Indica se o código irá possuir letras maiúsculas
        letras_minusculas (boolean): Indica se o código irá possuir letras minúsculas
        numeros (boolean): Indica se o código irá possuir números
        simbolos (boolean): Indica se o código irá possuir símbolos
        prefixo (string): prefixo para o código
        contador (inteiro): Contador para a quantidade de chamadas recursivas desta função
    
    Returns:
        Lista de códigos aleatórios gerados ou lista informando para tentar novamente com outros parâmetros
    '''
    
    # Cria a string que define os tipos de caracteres a serem usados no código
    caracteres_codigo = ''
    
    # Verifica os parâmetros dos tipos de caracteres e insere na string    
    if letras_maiusculas:
        caracteres_codigo += string.ascii_uppercase
    if letras_minusculas:
        caracteres_codigo += string.ascii_lowercase
    if numeros:
        caracteres_codigo += string.digits
    if simbolos:
        caracteres_codigo += string.punctuation
        
    # Retira os caracteres que podem gerar confusão no preenchimento dos votos
    caracteres_codigo = caracteres_codigo.replace("l", "")
    caracteres_codigo = caracteres_codigo.replace("1", "")
    caracteres_codigo = caracteres_codigo.replace("o", "")
    caracteres_codigo = caracteres_codigo.replace("0", "")
    
    # Cria a lista de códigos
    codigos = []
    
    # Itera sobre a quantidade, cria cada código e insere na lista de códigos
    for i in range(quantidade):
        codigo = ''.join(secrets.choice(caracteres_codigo) for i in range(tamanho))
        # Verifica se o código não existe na planilha dos códigos gerados originalmente
        if codigo not in codigos_existentes:
            codigos.append(prefixo + codigo)
    
    # Cria set para verificação da quantidade de códigos únicos
    teste = set(codigos)
    
    # Compara o tamanho do set de teste com a lista de códigos
    # Se algum código não for único ele chama recursivamente a mesma função para tentar de novo
    # Se tentou 10 vezes e não conseguiu gerar a quantidade de códigos únicos, retorna apenas a informação
    if len(teste) != len(codigos):
        if contador < 20:
            # Códigos não são únicos
            contador += 1
            gerar_codigos(quantidade, tamanho, letras_maiusculas, letras_minusculas, numeros, simbolos, prefixo, contador)
        return ['Não foi possível criar a quantidade de códigos aleatório com os parâmetros definidos, tente com maior variedade ou quantidade de caracteres']    
    
    # Se todos os códigos forem únicos, retorna a lista de códigos
    return codigos

def criar_excel(df, caminho='dt_pandas_to_excel.xlsx', indice=True, nome_planilha='Planilha1'):
    '''Cria um arquivo do excel formatado a partir de um data frame pandas
    
    Parameters:
        df (objeto pandas DataFrame): Data frame a ser convertido em planilha do excel
        caminho (string): Caminho do local para salvar o arquivo, com nome do arquivo e extensão
        indice (boolean): Indica se o arquivo irá ou não criar coluna para os índices do data frame
        nome_planilha (string): Nome para a aba/planilha da pasta de trabalho do excel
    
    Returns:
        Não possui retorno    
    '''
    
    # Cria um escritor pandas excel usando XlsxWriter como mecanismo e indicando o caminho como parâmetro
    escritor = pd.ExcelWriter(caminho, engine='xlsxwriter')

    # Converte o data frame em um objeto excel XlsxWriter
    df.to_excel(
        escritor,
        sheet_name=nome_planilha, # Define o nome da aba/planilha da pasta de trabalho
        startrow=1, # Pula uma linha para que seja possível inserir cabeçalho personalizado
        header=False, # Desativa o cabeçalho padrão
        index=indice # Define se irá criar coluna para índice ou não de acordo com o parâmtero informado
    )

    # Obtém a pasta de trabalho XlsxWriter e os objetos de planilha
    pasta_trabalho = escritor.book # Pasta de trabalho
    pasta_trabalho = escritor.sheets[nome_planilha] # Planilha

    # Obtém as dimensões do data frame
    (qtde_linhas, qtde_colunas) = df.shape

    # Cria uma lista de cabeçalhos das colunas para uso no add_table()
    config_colunas = []
    if indice == True: # Se Parâmetro do índice for verdadeiro, inclui o primeiro cabeçalho como 'Índice'
        config_colunas.append({'header': 'Índice'})
    for cabecalho in df.columns: # Para cada coluna no data frame, inclui o cabeçalho na lista
        config_colunas.append({'header': cabecalho})
    
    # Caso parâmetro do índice for falso, diminui uma coluna da quantidade de colunas 
    if indice == False:
        qtde_colunas -= 1
        
    # Adiciona a tabela
    pasta_trabalho.add_table(0, 0, qtde_linhas, qtde_colunas, {'columns': config_colunas})

    # Aumente a largura das colunas para melhor visualização
    pasta_trabalho.set_column(0, qtde_colunas, 25)

    # Fecha o escritor pandas excel writer e salva o arquivo excel
    escritor.save()

### Geração dos códigos de validação ->

In [19]:
# Lista de tuplas com a quantidade de novos códigos a serem gerados por categoria
qtde_codigos_categorias = [
    ('Confeitaria', 600),
    ('Pizzas', 1200),
]

# Cria variável com a quantidade total de códigos a serem gerados
qtde_total_codigos = 0
for codigo in qtde_codigos_categorias:
    qtde_total_codigos += codigo[1]

# Gera todos os códigos necessários e atribui a um data frame pandas
df_codigos = pd.DataFrame({'Código':gerar_codigos(qtde_total_codigos)})

# Cria a coluna categoria no data frame e atribui uma string vazia para todas as células
df_codigos['Categoria'] = ''

# Cria a coluna estabelecimento no data frame e atribui uma string vazia para todas as células
# Esta coluna será utilizada apenas após o data frame ser convertido em pasta de trabalho do excel
# Quando os códigos forem distribuídos aos estabelecimentos, será feita a anotação na planilha nesta coluna
df_codigos['Estabelecimento'] = ''

# Atribui a categoria à coluna categoria do data frame
# a quantidade de vezes da lista de quantidade de códigos por categoria
iaux = 0 # iterador auxiliar para contar os códigos por categoria
for codigo in qtde_codigos_categorias:
    df_codigos['Categoria'][iaux:iaux+codigo[1]] = codigo[0]
    iaux += codigo[1]

# Rearranja os índices a partir do valor 1
# para facilitar visualização e controle posterior no excel
df_codigos.index = np.arange(1, len(df_codigos)+1)

# Cria pasta de trabalho do excel com o data frame
criar_excel(df_codigos, caminho='codigos-categorias-geral-NOVO.xlsx')

# Cria uma lista de tuplas com as tabelas por categoria
# Primeiro índice da tupla recebe a categoria
# Segundo índice da tupla recebe um data frame com os códigos correspondentes à categoria
df_tabelas_categorias = [(categoria, df_codigos[df_codigos['Categoria'] == categoria]) for categoria, quantidade in qtde_codigos_categorias]

# Cria pastas de trabalho do excel com os data frames das categorias, um por um
for categoria, df in df_tabelas_categorias:
    # Cria uma planilha da categoria para controle dos códigos ao serem enviados aos estabelecimentos do evento
    criar_excel(df, caminho=r'codigos-controle-{}-NOVO.xlsx'.format(categoria))
    # Cria uma planilha da categoria apenas com os códigos para fins de impressão em gráfica
    # criar_excel(pd.DataFrame(df['Código']), caminho=r'codigos-impressão-{}.xlsx'.format(categoria), indice=False)